<font size="4">**Общая картина. Расчет плотности финансовых организаций (ФО) по регионам РФ и построение тепловой карты**</font>

In [184]:
import folium
from folium.plugins import HeatMap, FastMarkerCluster
import pandas as pd
import html

In [152]:
# Загрузка данных 
df = pd.read_excel('Свод ГО и подразделения.xlsx') 

In [315]:
# Преобразуем координаты в числовой формат
df['geo_lat'] = pd.to_numeric(df['geo_lat'], errors='coerce')
df['geo_lon'] = pd.to_numeric(df['geo_lon'], errors='coerce')

In [317]:
# Проверьте диапазон координат
print(f"Широта: {df['geo_lat'].min()} - {df['geo_lat'].max()}")
print(f"Долгота: {df['geo_lon'].min()} - {df['geo_lon'].max()}")

Широта: 41.461889 - 73.50673
Долгота: 19.894477 - 179.25444


In [319]:
# Создаем столбец с полным адресом
df['full_address'] = (df['federal_district'] + ' ФО, ' + df['region'] + ', ' + df['city'] + ', ' + df['street'] + ', ' + df['house'].astype(str))

In [321]:
# Создаем базовую карту
center_lat = (df['geo_lat'].min() + df['geo_lat'].max()) / 2
center_lon = (df['geo_lon'].min() + df['geo_lon'].max()) / 2

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=4,
    tiles='OpenStreetMap'
)

In [323]:
# 1. Добавляем тепловую карту (для общей плотности)
print("Создаем тепловую карту...")
heat_data = [[row['geo_lat'], row['geo_lon']] for _, row in df.iterrows()]

HeatMap(
    heat_data,
    radius=10,
    blur=15,
    max_zoom=12,
    min_opacity=0.3,
    gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'},
    name="Тепловая карта плотности"
).add_to(m)

Создаем тепловую карту...


In [324]:
# 2. Добавляем быстрые кластеры маркеров (для детальной информации)
print("Создаем кластеры маркеров...")

# Создаем список координат и popup-текстов
locations = []
popups = []

for _, row in df.iterrows():
    # Формируем popup текст в одну строку с разделителями
    popup_text = (
        f"{row['name']} | "
        f"ИНН: {row['inn']}"
        
    )
    
    # Обрезаем длинный текст (если нужно)
    if len(popup_text) > 100:
        popup_text = popup_text[:97] + "..."
    
    locations.append([row['geo_lat'], row['geo_lon']])
    popups.append(popup_text)

# Добавляем FastMarkerCluster
FastMarkerCluster(
    data=locations,
    popups=popups,
    name="Финансовые организации",
    options={
        'showCoverageOnHover': False,
        'spiderfyOnMaxZoom': True,
        'disableClusteringAtZoom': 15
    }
).add_to(m)

Создаем кластеры маркеров...


In [326]:
# Проверочный код для тестирования popups
# test_locations = [
#     [55.7558, 37.6173],  # Москва
#     [59.9343, 30.3351],  # СПб
#     [54.9893, 73.3682]   # Омск
# ]

# test_popups = [
#     "Тест 1: Сбербанк",
#     "Тест 2: ВТБ", 
#     "Тест 3: Газпромбанк"
# ]

# # Добавляем тестовые точки
# for i, (loc, popup) in enumerate(zip(test_locations, test_popups)):
#     folium.Marker(
#         location=loc,
#         popup=popup,
#         icon=folium.Icon(color='red', icon='info-sign')
#     ).add_to(m)

In [327]:
# Создаем FastMarkerCluster с кастомными иконками
def create_cluster_icon(feature):
    """Создаем иконки разного цвета в зависимости от типа организации"""
    color = 'green' if 'Кредит' in str(feature['properties']['tp']) else 'blue'
    return folium.Icon(color=color, icon='bank', prefix='fa')

In [331]:
# 3. Добавляем слой управления
folium.LayerControl(
    position='topright',
    collapsed=False
).add_to(m)

In [333]:
# 4. Добавляем заголовок
title_html = '''
<h3 align="center" style="font-size:16px; margin:10px;">
    <b>Плотность финансовых организаций по регионам РФ</b>
</h3>
<p align="center" style="font-size:12px; margin:5px;">
    Тепловая карта показывает плотность, кластеры - конкретные организации
</p>
'''
m.get_root().html.add_child(folium.Element(title_html))

In [335]:
# 5. Сохраняем карту
m.save('financial_organizations_complete_map.html')

print("✅ Карта успешно создана!")
print(f"📊 Организаций на карте: {len(df)}")
print(f"📍 Файл: financial_organizations_complete_map.html")

✅ Карта успешно создана!
📊 Организаций на карте: 48826
📍 Файл: financial_organizations_complete_map.html
